In [ ]:
# RUNNING FOR THE FIRST TIME: Load & cd into gdrive for persistent data
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/'My Drive'

# Download/Install/Configure CUDALucas
!wget https://raw.github.com/tdulcet/Linux-System-Information/master/info.sh -qO - | bash -s # Check System Info
!nvidia-smi --query-gpu=gpu_name --format=csv,noheader # Output what GPU is assigned to this Notebook
!wget https://raw.github.com/tdulcet/Distributed-Computing-Scripts/master/cudalucas.sh -q
!sed -i '/^GPU=/,/^fi/ s/^/# /' cudalucas.sh # Do not check for an Nvidia GPU
!sed -i '/^nohup / s/^/# /' cudalucas.sh # Do not start PrimeNet
!sed -i '/^crontab / s/^/# /' cudalucas.sh # Do not create a cronjob
!bash -- cudalucas.sh "" # Do not pass any additional parameters here
%cd cudalucas
!chmod -R 777 * # Gdrive permissions are strict, make them lax.
!nohup python primenet.py -d -T 100 -u USERNAME -p PASSWORD & # Start PrimeNet. Replace USERNAME, PASSWORD with your actual GIMPS Username & Password
!sleep 1
!sudo nice -n -20 ./CUDALucas # Start CUDALucas

In [ ]:
# RUNNING SUBSEQUENT TIMES: Restarting CUDALucas when Colab/you shut the process/notebook down
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/'My Drive'/cudalucas
!chmod -R 777 * # Gdrive permissions are strict, make them lax.
!nohup python primenet.py -d -T 100 -u USERNAME -p PASSWORD & # Start PrimeNet. Replace USERNAME, PASSWORD with your actual GIMPS Username & Password
!sleep 1
!sudo nice -n -20 ./CUDALucas # Start CUDALucas

In [1]:
# DEBUGGING
!nvidia-smi --query-gpu=gpu_name --format=csv,noheader # Output what GPU is assigned to this Notebook